# Draw the winding function of phase a

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cycler
import matplotlib.ticker as ticker
from fractions import Fraction
from scipy.integrate import quad

plt.rcParams.update({
    'figure.figsize': (6.69, 3.5),
    'figure.dpi': 400,
    'figure.constrained_layout.use': True,
    
    'axes.titlesize': 0,        # Standard-Titelgröße
    'axes.titlepad': 0,

    'axes.labelsize': 15,       # Standard-Achsenbeschriftungsgröße
    'xtick.labelsize': 11,      # Standardgröße der X-Achsen-Beschriftungen
    'ytick.labelsize': 11,      # Standardgröße der Y-Achsen-Beschriftungen

    'lines.linewidth': 2,       # Standard-Linienstärke
    'grid.linestyle': '-', 
    'lines.markersize': 8,      # Standard-Markierungsgröße

    'legend.fontsize': 15,      # Standardgröße der Legende
    'font.size': 15,
    
    'axes.labelpad': 6,
    'axes.prop_cycle': cycler.cycler(color=["#00A6D6", "#003082", "#00B5E2", "#000000"])
})

# Calculations

In [2]:
# --- Parameters ---
m_stator_phases = 3
p_pairs = 1                   # Number of pole pairs
N_s = 50                     # Number of turns in series per stator phase
r = 0.127                       # Stator radius in meters
l = 0.254                       # Stack length in meters
g = 5e-3                      # Air gap length in meters
mu_0 = 4 * np.pi * 1e-7        # Permeability of free space in H/m
I_amplitude = 55 * np.sqrt(2)  # Peak current in Amperes

def current_sa(wt):
    return I_amplitude * np.cos(wt)  

def current_sb(wt):
    return I_amplitude * np.cos(wt - 2*np.pi/3)

def current_sc(wt):
    return I_amplitude * np.cos(wt + 2*np.pi/3)

### Calculate the self inductance of the machine
$L_{xy} = \frac{\mu_0 r l}{g} \int_{0}^{2\pi} N_x N_y d\phi$

$L_{x} = \frac{\mu_0 r l}{g} \int_{0}^{2\pi} N_x N_x d\phi$

In [3]:
def winding_function(theta):
    return N_s/2 * np.cos(p_pairs * theta)

# Self indctance calculation aa
L_a = (mu_0 * r * l) / g * quad(lambda theta: winding_function(theta)**2, 0, 2*np.pi)[0]


# Mutual inductance calculation ab
L_ab = (mu_0 * r * l) / g * quad(lambda theta: winding_function(theta) * winding_function(theta - 2*np.pi/3), 0, 2*np.pi)[0]

print("--- Inductance Calculation ---")
print(f"Inductance L_a: {L_a:.6f} H")
print(f"Mutual inductance L_ab: {L_ab:.6f} H")

--- Inductance Calculation ---
Inductance L_a: 0.015919 H
Mutual inductance L_ab: -0.007959 H


### Calculate the phase flux linkage $\lambda_{sa}$ at $\omega t = 0$ and $\omega t = \frac{3}{2} \pi$
$ \lambda_a = (L_l + L_m) i_a - L_m/2 i_b - L_m/2 i_c$

can be written as matrix to look good in report

In [4]:
pos1 = 0
pos2 = 3/2 * np.pi

lambda_sa1 = current_sa(pos1) * (L_a + L_ab) + current_sb(pos1) * -L_ab/2 + current_sc(pos1) * -L_ab/2
lambda_sa2 = current_sa(pos2) * (L_a + L_ab) + current_sb(pos2) * -L_ab/2 + current_sc(pos2) * -L_ab/2

print(f"\n--- Flux Linkage Calculation ---")
print(f"Flux linkage λ_sa at θ = 0: {lambda_sa1:.6f} Wb-turns")
print(f"Flux linkage λ_sa at θ = 270°: {lambda_sa2:.6f} Wb-turns")



--- Flux Linkage Calculation ---
Flux linkage λ_sa at θ = 0: 0.309546 Wb-turns
Flux linkage λ_sa at θ = 270°: 0.000000 Wb-turns


### Calculate the space vector of the current at $\omega t = 0$ and $ \omega t = \frac{3}{2}$
$ \vec{i} = i e^{j\phi^*} = i_\alpha + ji_\beta = i \cos(\phi^*) + ji \sin(\phi^*) $

In [5]:
i1_a = I_amplitude * np.cos(pos1) + 1j* I_amplitude * np.sin(pos1)
i2_a = I_amplitude * np.cos(pos2) + 1j* I_amplitude * np.sin(pos2)

i1_b = I_amplitude * np.cos(pos1 - 2*np.pi/3) + 1j* I_amplitude * np.sin(pos1 - 2*np.pi/3)
i2_b = I_amplitude * np.cos(pos2 - 2*np.pi/3) + 1j* I_amplitude * np.sin(pos2 - 2*np.pi/3)

i1_c = I_amplitude * np.cos(pos1 + 2*np.pi/3) + 1j* I_amplitude * np.sin(pos1 + 2*np.pi/3)
i2_c = I_amplitude * np.cos(pos2 + 2*np.pi/3) + 1j* I_amplitude * np.sin(pos2 + 2*np.pi/3)


print(f"\n--- Current Space Vector Calculation ---")
print(f"Current Space Vector at θ = 0: {i1_a:.2f} A")
print(f"Current Space Vector at θ = 270°: {i2_a:.2f} A")




--- Current Space Vector Calculation ---
Current Space Vector at θ = 0: 77.78+0.00j A
Current Space Vector at θ = 270°: -0.00-77.78j A


### Calculate the space vector of the flux linkage at $\omega t = 0$ and $ \omega t = \frac{3}{2}$

$ \lambda_a = (L_l + L_m) \vec{i_a} - L_m/2 \vec{i_b} - L_m/2 \vec{i_c}$

In [11]:
lambda_sa1 = i1_a * (L_a + L_ab) + i1_b * -L_ab/2 + i1_c * -L_ab/2
lambda_sa2 = i2_a * (L_a + L_ab) + i2_b * -L_ab/2 + i2_c * -L_ab/2

# transform to exp form for better readability
def to_exp_str(c_num, label="λ"):
    """Returns a string formatted as Magnitude * exp(j * Angle_deg)"""
    mag = np.abs(c_num)
    # Convert radians to degrees for better human readability
    angle_deg = np.degrees(np.angle(c_num))
    
    # Clean up 'floating point noise' (e.g., 1.2e-16 becomes 0.0)
    if np.isclose(mag, 0): return f"{label} = 0"
    
    return f"{label} = {mag:.4f} * exp(j * {angle_deg:.1f}°)"

lambda_sa1_exp = to_exp_str(lambda_sa1, label="λ_sa at θ=0")
lambda_sa2_exp = to_exp_str(lambda_sa2, label="λ_sa at θ=270°")

print(f"\n--- Flux Linkage Calculation with Space Vectors ---")
print(f"Flux linkage λ_sa at θ = 0: {lambda_sa1:.6f} Wb-turns")
print(f"Flux linkage λ_sa at θ = 270°: {lambda_sa2:.6f} Wb-turns")
print(f"Flux linkage λ_sa at θ = 0 (exp form): {lambda_sa1_exp} Wb-turns")
print(f"Flux linkage λ_sa at θ = 270° (exp form): {lambda_sa2_exp} Wb-turns")


--- Flux Linkage Calculation with Space Vectors ---
Flux linkage λ_sa at θ = 0: 0.309546+0.000000j Wb-turns
Flux linkage λ_sa at θ = 270°: 0.000000-0.309546j Wb-turns
Flux linkage λ_sa at θ = 0 (exp form): λ_sa at θ=0 = 0.3095 * exp(j * 0.0°) Wb-turns
Flux linkage λ_sa at θ = 270° (exp form): λ_sa at θ=270° = 0.3095 * exp(j * -90.0°) Wb-turns


### Give possible $i_a$, $i_b$ and $i_c$ combinations for $\mathcal{F}_m = 0$

$\mathcal{F}_m = N_m/2 \cdot \frac{3}{2} (i_a + i_be^{-j\frac{2\pi}{3}} + i_ce^{j\frac{2\pi}{3}})$

This will be zero if the amplitudes of $i_a$, $i_b$ and $i_c$ are the same


### Given $i_{sa} = i_{sb} = i_{sc} = 55 \sqrt{2} \cos(\omega t) $ calculate $i_{s \alpha}$, $i_{s \beta}$ and $i_{s0}$ and the MMF space vetor $\vec{F_m}$ for $\omega t = 0$

In [8]:
def current_space_vector(wt):
    return 55*np.sqrt(2) * np.cos(wt)

I_amplitude = 55 * np.sqrt(2)  # Peak current in Amperes

# Transformation matrix for 3-phase to αβ transformation
T = (2/3) * np.array([[1, -0.5, -0.5],
                      [0, np.sqrt(3)/2, -np.sqrt(3)/2],
                      [1/2, 1/2, 1/2]])  # Added zero-sequence component for completeness

i_abc = np.array([I_amplitude, I_amplitude, I_amplitude])  # Apply to current aplitude for each phase
i_alpha_beta_zero = T @ i_abc

print(f"\n--- Clarke Transformation ---")
print(f"Current i_alpha: {i_alpha_beta_zero[0]} A")
print(f"Current i_beta: {i_alpha_beta_zero[1]} A")
print(f"Current i_zero: {i_alpha_beta_zero[2]} A")



--- Clarke Transformation ---
Current i_alpha: 0.0 A
Current i_beta: 0.0 A
Current i_zero: 77.78174593052023 A
